In [1]:
%load_ext autoreload

In [2]:
import numpy as np
from functools import partial
import visualizations_utils as viz_utils
from iris_plant_visualizer import IrisPlantVisualizer
import ipywidgets as widgets
from IPython.display import display
from scipy.linalg import block_diag
import matplotlib.pyplot as plt
import cdd

In [3]:
#pydrake imports
from pydrake.common import FindResourceOrThrow
from pydrake.multibody.parsing import Parser
from pydrake.multibody.plant import AddMultibodyPlantSceneGraph
from pydrake.systems.framework import DiagramBuilder
from pydrake.geometry import Role, GeometrySet, CollisionFilterDeclaration
from pydrake.solvers import mathematicalprogram as mp
from pydrake.all import RigidTransform, RollPitchYaw, RevoluteJoint
from pydrake.all import RotationMatrix
import time
import pydrake.multibody.rational_forward_kinematics as rational_forward_kinematics
from pydrake.all import RationalForwardKinematics
from pydrake.geometry.optimization import IrisOptionsRationalSpace, IrisInRationalConfigurationSpace, HPolyhedron, Hyperellipsoid

# Build and set up the visualization the plant and the visualization of the C-space obstacle

Note that running this cell multiple times will establish multiple meshcat instances which can fill up your memory. It is a good idea to call "pkill -f meshcat" from the command line before re-running this cell


In [4]:
builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
parser = Parser(plant)
oneDOF_iiwa_asset = FindResourceOrThrow("drake/C_Iris_Examples/assets/oneDOF_iiwa7_with_box_collision.sdf")

box_asset = FindResourceOrThrow("drake/C_Iris_Examples/assets/box_small.urdf")

models = []
models.append(parser.AddModelFromFile(box_asset))
models.append(parser.AddModelFromFile(oneDOF_iiwa_asset, 'right_sweeper'))
models.append(parser.AddModelFromFile(oneDOF_iiwa_asset, 'left_sweeper'))




locs = [[0.,0.,0.],
        [0,1,0.85],
        [0,-1,0.55]]
plant.WeldFrames(plant.world_frame(), 
                 plant.GetFrameByName("base", models[0]),
                 RigidTransform(locs[0]))

t1 = RigidTransform(RollPitchYaw([np.pi/2, 0, 0]).ToRotationMatrix(), locs[1])@RigidTransform(RollPitchYaw([0, 0, np.pi/2]), np.zeros(3))
t2 = RigidTransform(RollPitchYaw([-np.pi/2, 0, 0]).ToRotationMatrix(), locs[2])@RigidTransform(RollPitchYaw([0, 0, np.pi/2]), np.zeros(3))
plant.WeldFrames(plant.world_frame(), 
                 plant.GetFrameByName("iiwa_oneDOF_link_0", models[1]), 
                 t1)
plant.WeldFrames(plant.world_frame(), 
                 plant.GetFrameByName("iiwa_oneDOF_link_0", models[2]), 
                 t2)


plant.Finalize()

idx = 0
q0 = [0.0, 0.0]
# q_low  = [-1.7, -1.7, 0]
# q_high = [1.7, 1.7,  0]
eps = 0# 1e-3
val = 1.7
q_low  = [-val+eps, -val+eps, 0]
q_high = [val-eps, val-eps,  0]
# set the joint limits of the plant
for model in models:
    for joint_index in plant.GetJointIndices(model):
        joint = plant.get_mutable_joint(joint_index)
        if isinstance(joint, RevoluteJoint):
            joint.set_default_angle(q0[idx])
            joint.set_position_limits(lower_limits= np.array([q_low[idx]]), upper_limits= np.array([q_high[idx]]))
            idx += 1
        
# construct the RationalForwardKinematics of this plant. This object handles the
# computations for the forward kinematics in the tangent-configuration space
Ratfk = RationalForwardKinematics(plant)

# the point about which we will take the stereographic projections
q_star = np.zeros(3)

#compute limits in t-space
limits_t = []
for q in [q_low, q_high]:
    limits_t.append(Ratfk.ComputeTValue(np.array(q), q_star)[:2])

do_viz = True

# This line builds the visualization. Change the viz_role to Role.kIllustration if you
# want to see the plant with its illustrated geometry or to Role.kProximity
visualizer = IrisPlantVisualizer(plant, builder, scene_graph, viz_role=Role.kIllustration)
diagram = visualizer.diagram

# This line will run marching cubes to generate a mesh of the C-space obstacle
# Increase N to increase the resolution of the C-space obstacle.


You can open the visualizer by visiting the following URL:
http://127.0.0.1:7002/static/
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7003/static/
Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6002...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7002/static/
Connected to meshcat-server.


In [5]:
visualizer.visualize_collision_constraint(N = 50, factor = 1, iso_surface = 0.5, wireframe = False)

In [6]:


translation = RigidTransform(RotationMatrix(np.eye(3)), np.array([0,2,0]))

camera_transform = translation.GetAsMatrix4()
visualizer.vis2["/Cameras/default/rotated/<object>"].set_transform(camera_transform)

# n = 1000
# for i in range(n):
#     theta = 2*i/n*np.pi
    
#     translation = RigidTransform(RotationMatrix(np.eye(3)), np.array([0,theta,0]))

#     camera_transform = translation.GetAsMatrix4()
#     visualizer.vis2["/Cameras/default/rotated/<object>"].set_transform(camera_transform)

    
#     rot = RigidTransform(RotationMatrix.MakeZRotation(i),
#                          np.zeros(3))


#     translation = RigidTransform(RotationMatrix(np.eye(3)), np.array([0,0,100]))

#     camera_transform = (rot.multiply(translation)).GetAsMatrix4()
#     # print(camera_transform)
    
# #     visualizer.vis2["/Cameras/default/rotated/<object>"].set_transform(camera_transform)
#     visualizer.vis["/Cameras/default/"].set_transform(camera_transform)
#     time.sleep(3/n)
    
# visualizer.vis2["/Axes"].delete()
# visualizer.vis2["/Grid"].delete()
# visualizer.vis2["/Background"].delete()
# visualizer.visualize_collision_constraint(N = 30, factor = 2, iso_surface = 0.5, wireframe = False)

## Set up the sliders so we can move the plant around manually

In [7]:
sliders = []
sliders.append(widgets.FloatSlider(min=q_low[0], max=q_high[0], value=0, description='q0'))
sliders.append(widgets.FloatSlider(min=q_low[1], max=q_high[1], value=0, description='q1'))
# sliders.append(widgets.FloatSlider(min=q_low[2], max=q_high[2], value=0, description='q2'))

q = q0.copy()
def handle_slider_change(change, idx):
    q[idx] = change['new']
    #print(q, end="\r")
    visualizer.showres(q)
    visualizer.visualize_planes()
    
idx = 0
for slider in sliders:
    slider.observe(partial(handle_slider_change, idx = idx), names='value')
    idx+=1

for slider in sliders:
    display(slider)


visualizer.jupyter_cell()

Widget Javascript not detected.  It may not be installed or enabled properly.


Widget Javascript not detected.  It may not be installed or enabled properly.


In [8]:
# filter fused joints self collisions so they don't interfere with collision engine
digaram = visualizer.diagram
context = visualizer.diagram_context
sg_context = scene_graph.GetMyContextFromRoot(context)
inspector = scene_graph.model_inspector()

pairs = scene_graph.get_query_output_port().Eval(sg_context).inspector().GetCollisionCandidates()

gids = [gid for gid in inspector.GetGeometryIds(GeometrySet(inspector.GetAllGeometryIds()), Role.kProximity)]
get_name_of_gid = lambda gid : inspector.GetName(gid)
gids.sort(key=get_name_of_gid)
right_sweeper_gids = [gid for gid in gids if "right_sweeper::" in get_name_of_gid(gid)]
left_sweeper_gids = [gid for gid in gids if "left_sweeper::" in get_name_of_gid(gid)]

right_sweeper_fused_col_geom = right_sweeper_gids[2:]
right_sweeper_fused_set = GeometrySet(right_sweeper_fused_col_geom)
left_sweeper_fused_col_geom = left_sweeper_gids[4:]
left_sweeper_fused_set = GeometrySet(left_sweeper_fused_col_geom)
scene_graph.collision_filter_manager()\
            .Apply(CollisionFilterDeclaration().ExcludeWithin(right_sweeper_fused_set))
scene_graph.collision_filter_manager()\
            .Apply(CollisionFilterDeclaration().ExcludeWithin(left_sweeper_fused_set))

right_sweeper_end_gid = right_sweeper_gids[-1]
left_sweeper_end_gid = left_sweeper_gids[-1]
id_pairs_of_interest = [(right_sweeper_end_gid, left_sweeper_end_gid),
                       ]
visualizer.collision_pairs_of_interest = id_pairs_of_interest


# Setup IRIS Options and Generate Regions

In [9]:
# Some examples of some seed points which give large regions using the
# non-linear programming approach to IRIS described in Algorithm 3
seed_points_q = np.array([   [0.0, 0],
                              [0.7, -0.9],
                              [-0.5, -0.5],
                              [0.4,-1.3]
                              ])
seed_points = np.array([Ratfk.ComputeTValue(seed_points_q[idx], np.zeros((2,)))\
                        for idx in range(seed_points_q.shape[0])])
if do_viz:
    visualizer.plot_seedpoints(seed_points)


In [10]:
regions = []
ellipses = []

iris_options = IrisOptionsRationalSpace()
iris_options.require_sample_point_is_contained = True
iris_options.iteration_limit = 20
iris_options.configuration_space_margin = 1e-5
iris_options.max_faces_per_collision_pair = 60
iris_options.termination_threshold = -1
iris_options.q_star = np.zeros(2)
iris_options.relative_termination_threshold = 0.05
iris_options.enable_ibex = False

def promote_region_to_3d(region, width = 0.2):
    A = block_diag(region.A(), np.array([-1,1])[:, np.newaxis])
    b = np.append(region.b(), width*np.ones(2))

    return HPolyhedron(A,b)
    
regions_3d = []

for i, s in enumerate(seed_points):
    q = Ratfk.ComputeQValue(s, np.zeros((2,)))
    plant.SetPositions(plant.GetMyMutableContextFromRoot(context), q)
    r = IrisInRationalConfigurationSpace(plant, plant.GetMyContextFromRoot(context), iris_options)
    regions.append(r)
    regions_3d.append(promote_region_to_3d(r))
    ellipses.append(r.MaximumVolumeInscribedEllipsoid())
if do_viz:
    visualizer.plot_regions(regions_3d,
                            ellipses=None,
                            region_suffix='_original',
                            randomize_colors = True,
                           wireframe = False)

# Certify regions

In [11]:
cspace_free_region_certifier = rational_forward_kinematics.CspaceFreeRegion(diagram, plant, scene_graph,
                                   rational_forward_kinematics.SeparatingPlaneOrder.kAffine,
                                   rational_forward_kinematics.CspaceRegionType.kGenericPolytope)
filtered_collision_pairs = set()
solver_options = mp.SolverOptions()
# make the solver verbose
solver_options.SetOption(mp.CommonSolverOption.kPrintToConsole, 0)

## first search for a minimal, uniform contraction of a proposed region which can be certified

In [12]:
# i = 0
# bisect_regions = []
# bisect_solutions = []
# for i in range(len(regions)):
#     print(i)
#     region_to_certify = regions[i]
#     seed_point = seed_points[i,:]


#     binary_search_options = rational_forward_kinematics.BinarySearchOption()
#     binary_search_options.epsilon_max = 0 # it is very unlikely that we can find a uniform expansion of the current region
#     binary_search_options.max_iters = 15
#     # speed up the bisection search by taking non-uniform steps when possible
#     binary_search_options.search_d = True
#     # find the smallest e such that At <= b + e1 still contains our seed point.
#     binary_search_options.epsilon_min = rational_forward_kinematics.FindEpsilonLower(region_to_certify.A(),
#                                                                                      region_to_certify.b(),
#                                                                                      limits_t[0], limits_t[1],
#                                                                                      seed_point)
#     #use as many threads as possible to speed up computation
# #     binary_search_options.num_threads = 1

#     certified_region_contraction_solution = cspace_free_region_certifier.CspacePolytopeBinarySearch(
#                                                                      q_star[:2],
#                                                                      filtered_collision_pairs,
#                                                                      region_to_certify.A(),
#                                                                      region_to_certify.b(),
#                                                                      binary_search_options, 
#                                                                      solver_options,
#                                                                      seed_point)
#     bisect_solutions.append(certified_region_contraction_solution)
#     certified_region_contraction = HPolyhedron(certified_region_contraction_solution.C,
#                                                certified_region_contraction_solution.d)
    
#     bisect_regions.append(promote_region_to_3d(certified_region_contraction))
# if do_viz:
#     visualizer.plot_regions(bisect_regions, ellipses=None,
#                             region_suffix='_certified_region_contraction',
#                             randomize_colors = True,
#                             wireframe = False)

In [13]:


            
# visualizer.certified_region_solution_list = bisect_solutions

In [31]:
import cdd
def regular_n_gon(n, center = np.zeros(2)):
    assert n > 2, "need n to be more than 2"
    verts = np.zeros((n,2))
    for i in range(n):
        verts[i] = np.array([np.cos(i*2*np.pi/n), np.sin(i* 2*np.pi/n)])
    return verts + center

def regular_n_gon_drake(n, center = np.zeros(2)):
    verts = regular_n_gon(n, center)
    verts_cdd = np.hstack([np.ones((verts.shape[0],1)), verts])
    cdd_mat = cdd.Matrix(verts_cdd, number_type = 'float')
    cdd_mat.rep_type = cdd.RepType.GENERATOR
    p = cdd.Polyhedron(cdd_mat)
    hrep = p.get_inequalities()
    A = np.zeros((hrep.row_size, 2))
    b = np.zeros(hrep.row_size)
    for i in range(hrep.row_size):
        A[i] = -np.array(hrep[i][1:])
        norm = np.linalg.norm(A[i])
        A[i] = A[i]/norm
        b[i] = hrep[i][0]/norm
    return HPolyhedron(A,b)

def perturbed_regular_n_gon_drake(n, center = np.zeros(2)):
    n_gon = regular_n_gon_drake(n, center)
    A = n_gon.A() + 1e-3*np.random.randn(*(n_gon.A().shape))
    norms = np.linalg.norm(A, ord=2, axis=1, keepdims=True)
    A = A/norms
    b = n_gon.b()/norms.flatten()
    return HPolyhedron(A,b)
    

tri = regular_n_gon_drake(5)
# verts = regular_n_gon(5)
# print(verts.shape)
# verts_cdd = np.hstack([np.ones((verts.shape[0],1)), verts])

In [32]:
i = 0
bilinear_regions = {i: [] for i in range(len(seed_points))}
bilinear_solutions =[]

num_faces = 10

max_iters = 20
bilinear_alternation_options = rational_forward_kinematics.BilinearAlternationOption()

bilinear_alternation_options.max_iters = 2
bilinear_alternation_options.lagrangian_backoff_scale = 0
bilinear_alternation_options.polytope_backoff_scale = 0
# number of threads used to solve this optimization.
bilinear_alternation_options.num_threads = 1

for i in range(len(seed_points)):
    seed_point = seed_points[i,:]
    
    region_to_certify = perturbed_regular_n_gon_drake(num_faces, seed_point[:2])
    
#     eps = rational_forward_kinematics.FindEpsilonLower(region_to_certify.A(),
#                                                  region_to_certify.b(),
#                                                  limits_t[0], limits_t[1],
#                                                  seed_point)
    eps = rational_forward_kinematics.FindEpsilonLowerVector(region_to_certify.A(),
                                                 region_to_certify.b(),
                                                 limits_t[0], limits_t[1],
                                                 seed_point)
    
    
    region_to_certify = HPolyhedron(region_to_certify.A(), region_to_certify.b()+ eps)
    for j in range(max_iters):
        sol, _, _ = cspace_free_region_certifier.CspacePolytopeBilinearAlternation(q_star[:2],
                                                                         filtered_collision_pairs,
                                                                         region_to_certify.A(),
                                                                         region_to_certify.b(),
                                                                         bilinear_alternation_options, 
                                                                         solver_options,
                                                                         seed_point)
        region_to_certify = HPolyhedron(sol.C, sol.d)
        bilinear_regions[i].append(promote_region_to_3d(region_to_certify))
#     bilinear_solutions.append(sol)
        
        
#     final_certified_region = HPolyhedron(certified_region_final_solution.C, certified_region_final_solution.d)
# if do_viz:
#     visualizer.plot_regions([final_certified_region], ellipses=None,
#                             region_suffix='_certified_region_final',
#                             randomize_colors = True)
#     #use as many threads as possible to speed up computation
# #     binary_search_options.num_threads = 1

#     certified_region_contraction_solution = cspace_free_region_certifier.CspacePolytopeBinarySearch(
#                                                                      q_star[:2],
#                                                                      filtered_collision_pairs,
#                                                                      region_to_certify.A(),
#                                                                      region_to_certify.b(),
#                                                                      binary_search_options, 
#                                                                      solver_options,
#                                                                      seed_point)
#     bisect_solutions.append(certified_region_contraction_solution)
#     certified_region_contraction = HPolyhedron(certified_region_contraction_solution.C,
#                                                certified_region_contraction_solution.d)

INFO:drake:Found Lagrangian multiplier and separating planes
INFO:drake:Lagrangian step time 11.332 s
INFO:drake:max(power(det(P), 1/2))=-nan, solver_time 0.015096902847290039
INFO:drake:cost improvement -nan
INFO:drake:mosek info 0, UnknownError
INFO:drake:Found Lagrangian multiplier and separating planes
INFO:drake:Lagrangian step time 11.897 s
INFO:drake:max(power(det(P), 1/2))=-nan, solver_time 0.010648012161254883
INFO:drake:cost improvement -nan
INFO:drake:mosek info 0, UnknownError
INFO:drake:Found Lagrangian multiplier and separating planes
INFO:drake:Lagrangian step time 10.767 s
INFO:drake:max(power(det(P), 1/2))=-nan, solver_time 0.008291006088256836
INFO:drake:cost improvement -nan
INFO:drake:mosek info 0, UnknownError
INFO:drake:Found Lagrangian multiplier and separating planes
INFO:drake:Lagrangian step time 10.135 s
INFO:drake:max(power(det(P), 1/2))=-nan, solver_time 0.019236087799072266
INFO:drake:cost improvement -nan
INFO:drake:mosek info 0, UnknownError
INFO:drake:F

INFO:drake:max(power(det(P), 1/2))=0.0036454211157178167, solver_time 0.004297018051147461
INFO:drake:Found Lagrangian multiplier and separating planes
INFO:drake:Lagrangian step time 11.805 s
INFO:drake:max(power(det(P), 1/2))=0.003645421107222218, solver_time 0.006905794143676758
INFO:drake:cost improvement inf
INFO:drake:Iter: 0, polytope step cost -0.009835596304324557, solver time 0.21848297119140625
INFO:drake:Found Lagrangian multiplier and separating planes
INFO:drake:Lagrangian step time 10.114 s
INFO:drake:max(power(det(P), 1/2))=0.004315752507831824, solver_time 0.007033109664916992
INFO:drake:cost improvement 0.0006703314006096064
INFO:drake:Iter: 1, polytope step cost -0.010003687355372015, solver time 0.23454904556274414
INFO:drake:max(power(det(P), 1/2))=0.005014668559280012, solver_time 0.005712985992431641
INFO:drake:Found Lagrangian multiplier and separating planes
INFO:drake:Lagrangian step time 12.243 s
INFO:drake:max(power(det(P), 1/2))=0.005014668533293625, solver

INFO:drake:Found Lagrangian multiplier and separating planes
INFO:drake:Lagrangian step time 13.945 s
INFO:drake:max(power(det(P), 1/2))=0.02310528215264461, solver_time 0.0063059329986572266
INFO:drake:cost improvement inf
INFO:drake:Iter: 0, polytope step cost -0.029846164368544297, solver time 0.4060039520263672
INFO:drake:Found Lagrangian multiplier and separating planes
INFO:drake:Lagrangian step time 12.134 s
INFO:drake:max(power(det(P), 1/2))=0.023980973368264217, solver_time 0.006421089172363281
INFO:drake:cost improvement 0.0008756912156196069
INFO:drake:Iter: 1, polytope step cost -0.030439771865456742, solver time 0.4120371341705322
INFO:drake:max(power(det(P), 1/2))=0.024838884790357185, solver_time 0.004268169403076172
INFO:drake:Found Lagrangian multiplier and separating planes
INFO:drake:Lagrangian step time 12.571 s
INFO:drake:max(power(det(P), 1/2))=0.024838884790336538, solver_time 0.005527019500732422
INFO:drake:cost improvement inf
INFO:drake:mosek info 5, Infeasibl

INFO:drake:Found Lagrangian multiplier and separating planes
INFO:drake:Lagrangian step time 11.863 s
INFO:drake:max(power(det(P), 1/2))=0.11100851119562762, solver_time 0.01872992515563965
INFO:drake:cost improvement inf
INFO:drake:Iter: 0, polytope step cost -0.04134017122485413, solver time 0.42382311820983887
INFO:drake:Found Lagrangian multiplier and separating planes
INFO:drake:Lagrangian step time 12.072 s
INFO:drake:max(power(det(P), 1/2))=0.11707763781194777, solver_time 0.00952291488647461
INFO:drake:cost improvement 0.006069126616320156
INFO:drake:Iter: 1, polytope step cost -0.04168600455213127, solver time 0.29236793518066406
INFO:drake:max(power(det(P), 1/2))=0.12304266945536453, solver_time 0.005015850067138672
INFO:drake:Found Lagrangian multiplier and separating planes
INFO:drake:Lagrangian step time 10.933 s
INFO:drake:max(power(det(P), 1/2))=0.12304266945514446, solver_time 0.004807949066162109
INFO:drake:cost improvement inf
INFO:drake:Iter: 0, polytope step cost -0

INFO:drake:max(power(det(P), 1/2))=0.22143660595723547, solver_time 0.005742073059082031
INFO:drake:cost improvement inf
INFO:drake:Iter: 0, polytope step cost -0.045887309932305144, solver time 0.3744971752166748
INFO:drake:Found Lagrangian multiplier and separating planes
INFO:drake:Lagrangian step time 11.04 s
INFO:drake:max(power(det(P), 1/2))=0.22536199134290075, solver_time 0.005976200103759766
INFO:drake:cost improvement 0.003925385385665281
INFO:drake:Iter: 1, polytope step cost -0.045811675693091275, solver time 0.2490828037261963
INFO:drake:max(power(det(P), 1/2))=0.229177436423013, solver_time 0.004630088806152344
INFO:drake:Found Lagrangian multiplier and separating planes
INFO:drake:Lagrangian step time 10.089 s
INFO:drake:max(power(det(P), 1/2))=-nan, solver_time 0.014093875885009766
INFO:drake:cost improvement -nan
INFO:drake:mosek info 0, UnknownError
INFO:drake:Found Lagrangian multiplier and separating planes
INFO:drake:Lagrangian step time 12.432 s
INFO:drake:max(pow

In [33]:
for j, (i,r) in enumerate(bilinear_regions.items()):
    if do_viz:
        visualizer.plot_regions(r, ellipses=None,
                            region_suffix=f'_regions_seedpoint_{i}',
                            randomize_colors = True)

In [17]:
# for i in range(len(seed_points)):
#     seed_point = seed_points[i,:]
    
#     region_to_certify = regular_n_gon_drake(num_faces, seed_point[:2])
#     eps = rational_forward_kinematics.FindEpsilonLowerVector(region_to_certify.A(),
#                                                  region_to_certify.b(),
#                                                  limits_t[0], limits_t[1],
#                                                  seed_point)
    
#     region_to_certify = promote_region_to_3d(HPolyhedron(region_to_certify.A(), region_to_certify.b()+ 0.8*eps))
    
#     visualizer.plot_regions([region_to_certify], ellipses=None,
#                             region_suffix=f'_regions_start_{i}',
#                             randomize_colors = True)